In [227]:
import cv2,numpy
height,width = 480,720
image = 100*numpy.ones((height,width), numpy.uint8)
image = cv2.rectangle(image, (width//4,height//4), (3*width//4,3*height//4), color=(255,255,255), thickness=-1)
   
# Displaying the image
#cv2.namedWindow('Orginal', cv2.WINDOW_NORMAL)
cv2.imshow('Orginal', image) 
cv2.waitKey(0)
cv2.destroyAllWindows()

# Warp Affine with bilinear interpolation

In [257]:
from math import floor,ceil,sqrt,cos,sin,pi

theta = 0
Ar = numpy.array([[cos(theta),-sin(theta)],[sin(theta),cos(theta)]]) 

s1 = 2
s2 = 2
Asc = numpy.array([[s1,0],[0,s2]])

S = 1
Ash = numpy.array([[1,0],[S,1]])

A = (Ar.dot(Asc)).dot(Ash)

#A = numpy.array([[1,-1],[1,1]]) 
Ainv = numpy.linalg.inv(A)
center = numpy.array([height/2,width/2])
b = center-A.dot(center)
b += numpy.array([0,0])



registImg = numpy.zeros((height,width), numpy.uint8)
for i in range(height):
    for j in range(width):
        #coList = Ainv.dot(numpy.array([i,j])-b)
        coList = A.dot(numpy.array([i,j]))+b
        x,y = coList[0],coList[1]
        
        #Interpolation
        if x>=0 and x<=height-1 and y>=0 and y<=width-1:            
            cx = ceil(x)
            fx = floor(x)
            cy = ceil(y)
            fy = floor(y)
            if cx==fx:
                dxf = 1/2
                dxb = 1/2
            else:
                dxf = cx-x
                dxb = x-fx
            if cy==fy:
                dyf = 1/2
                dyb = 1/2
            else:
                dyf = cy-y
                dyb = y-fy
            registImg[i][j] = dyb*dxb*image[cx][cy]+dyb*dxf*image[fx][cy]+dyf*dxb*image[cx][fy]+dyf*dxf*image[fx][fy]
            
#cv2.namedWindow('RegistImg', cv2.WINDOW_NORMAL)
cv2.imshow('RegistImg', registImg) 
cv2.waitKey(0)
cv2.destroyAllWindows()

# Vector Field

In [258]:
nx = 10
ny = 15
hx = height//nx
hy = width//ny
px = []
py = []
tx = []
ty = []
vecField = numpy.zeros((height,width))
for i in range(1,nx):
    for j in range(1,ny):
        px.append(i*hx)
        py.append(j*hy)
        #dstP = A.dot(numpy.array([i*hx,j*hy]))+b
        dstP = Ainv.dot(numpy.array([i*hx,j*hy])-b)
        tx.append(dstP[0])
        ty.append(dstP[1])
        
px = numpy.array(px)
py = numpy.array(py)
tx = numpy.array(tx)
ty = numpy.array(ty)

mt = ((tx-px)**2+(ty-py)**2)**(1/2)
maxt = numpy.max(mt)/numpy.min([hx,hy])

for k in range(0,(nx-1)*(ny-1)):
        cv2.arrowedLine(vecField,(py[k],px[k]),(floor(py[k]+0.5+(ty[k]-py[k])/maxt),floor(px[k]+0.5+(tx[k]-px[k])/maxt)),(255,255,255),1,8,0,0.2)
        
#cv2.namedWindow('Field', cv2.WINDOW_NORMAL)
cv2.imshow('Field', vecField) 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [259]:
com = numpy.vstack((registImg,numpy.uint8(vecField)))
cv2.namedWindow('com', cv2.WINDOW_NORMAL)
cv2.imshow('com', com) 
cv2.waitKey(0)
cv2.destroyAllWindows()